In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
import xgboost as xgb
import seaborn as sns

Using TensorFlow backend.


In [2]:
df = pd.read_csv('fightdata.csv')

In [3]:
df.head(10)

BPrev  BStreak  B_Age  B_Height                        B_HomeTown  B_ID  \
0      0        0   38.0     193.0                  Hounslow England   808   
1      0        0   36.0     172.0   Chicago, Illinois United States  1054   
2      0        0   39.0     167.0       Isla Vista , California USA   959   
3      0        0   33.0     167.0                 San Diego, CA USA  1056   
4      0        0   36.0     185.0               Southampton England  2005   
5      0        0   35.0     180.0                   Amazonas Brazil  2101   
6      0        0   26.0     175.0   Detroit, Michigan United States  2157   
7      0        0   33.0     175.0                   Dagestan Russia  2158   
8      0        0   36.0     180.0  Zinzeli, Astrakhan Oblast Russia  2160   
9      0        0   33.0     162.0          Dagestan Republic Russia  2084   

                             B_Location            B_Name  B_Weight  \
0             Amsterdam The Netherlands  Alistair Overeem     120.0   
1       Chicago, Illinois United States     Ricardo Lamas      65.0   
2            Sacramento, California USA      Urijah Faber      61.0   
3                     San Diego, CA USA    Danny Martinez      56.0   
4                   Southampton England        Tom Watson      84.0   
5                 Rio de Janeiro Brazil      Alan Patrick      70.0   
6       Las Vegas, Nevada United States         Kevin Lee      70.0   
7  Coconut Creek, Florida United States  Rashid Magomedov      70.0   
8      Zinzeli, Astrakhan Oblast Russia    Gasan Umalatov      77.0   
9              Dagestan Republic Russia    Ali Bagautinov      56.0   

   B__Round1_Grappling_Reversals_Landed  B__Round1_Grappling_Standups_Landed  \
0                                   NaN                                  NaN   
1                                   NaN                                  NaN   
2                                   NaN                                  NaN   
3                                   NaN                                  NaN   
4                                   NaN                                  NaN   
5                                   NaN                                  NaN   
6                                   NaN                                  NaN   
7                                   NaN                                  NaN   
8                                   NaN                                  NaN   
9                                   NaN                                  NaN   

   B__Round1_Grappling_Submissions_Attempts  \
0                                       NaN   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   
5                                       NaN   
6                                       NaN   
7                                       NaN   
8                                       NaN   
9                                       NaN   

   B__Round1_Grappling_Takedowns_Attempts  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   
5                                     NaN   
6                                     NaN   
7                                     NaN   
8                                     NaN   
9                                     NaN   

   B__Round1_Grappling_Takedowns_Landed  \
0                                   NaN   
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   
5                                   NaN   
6                                   NaN   
7                                   NaN   
8                                   NaN   
9                                   N

In [4]:
df.describe()

BPrev      BStreak        B_Age     B_Height         B_ID  \
count  2318.000000  2318.000000  2301.000000  2301.000000  2318.000000   
mean      2.391286     0.744607    31.776184   177.327249  2120.001726   
std       2.539978     1.145596     4.165267     8.807620   705.089725   
min       0.000000     0.000000    20.000000   152.000000   129.000000   
25%       0.000000     0.000000    29.000000   172.000000  1910.250000   
50%       2.000000     0.000000    31.000000   177.000000  2230.000000   
75%       4.000000     1.000000    35.000000   182.000000  2709.000000   
max      14.000000    10.000000    48.000000   213.000000  3196.000000   

          B_Weight  B__Round1_Grappling_Reversals_Landed  \
count  2306.000000                           1647.000000   
mean     73.699480                              0.074681   
std      15.425347                              0.305691   
min      52.000000                              0.000000   
25%      61.000000                              0.000000   
50%      70.000000                              0.000000   
75%      84.000000                              0.000000   
max     120.000000                              3.000000   

       B__Round1_Grappling_Standups_Landed  \
count                          1647.000000   
mean                              1.103825   
std                               1.537946   
min                               0.000000   
25%                               0.000000   
50%                               1.000000   
75%                               2.000000   
max                              13.000000   

       B__Round1_Grappling_Submissions_Attempts  \
count                               1647.000000   
mean                                   0.577413   
std                                    1.049758   
min                                    0.000000   
25%                                    0.000000   
50%                                    0.000000   
75%                                    1.000000   
max                                    8.000000   

       B__Round1_Grappling_Takedowns_Attempts  \
count                             1647.000000   
mean                                 3.852459   
std                                  5.261864   
min                                  0.000000   
25%                                  0.000000   
50%                                  2.000000   
75%                                  5.000000   
max                                 47.000000   

       B__Round1_Grappling_Takedowns_Landed  \
count                           1647.000000   
mean                               1.657559   
std                                2.443886   
min                                0.000000   
25%                                0.000000   
50%                                1.000000   
75%                                2.000000   
max                               20.000000   

       B__Round1_Strikes_Body Significant Strikes_Attempts  \
count                                        1647.000000     
mean                                           15.221615     
std                                            16.047585     
min                                             0.000000     
25%                                             4.000000     
50%                                            10.000000     
75%                                            21.000000     
max                                           124.000000     

       B__Round1_Strikes_Body Significant Strikes_Landed  \
count                                        1647.000000   
mean                                           10.128719   
std                                            10.967552   
min                                             0.000000   
25%                                             3.000000   
50%                                             7.000000   
75%                                            14.000000   
max                      

In [5]:
fights = df[['B_Name', 'B_ID', 'R_Name', 'R_ID', 'winby', 'winner', 'Date', 'Event_ID', 'Fight_ID', 'Max_round', 'Last_round']]
fights

,B_Name,B_ID,R_Name,R_ID,winby,winner,Date,Event_ID,Fight_ID,Max_round,Last_round
0,Alistair Overeem,808,Frank Mir,377,DEC,blue,02/03/2014,646,4580,3,3
1,Ricardo Lamas,1054,Jose Aldo,1052,DEC,red,02/03/2014,646,4589,5,5
2,Urijah Faber,959,Renan Barao,1527,KO/TKO,red,02/03/2014,646,4590,5,1
3,Danny Martinez,1056,Chris Cariaso,1253,DEC,red,02/03/2014,646,4605,3,3
4,Tom Watson,2005,Nick Catone,464,DEC,red,02/03/2014,646,4631,3,3
5,Alan Patrick,2101,John Makdessi,1675,DEC,blue,02/03/2014,646,4642,3,3
6,Kevin Lee,2157,Al Iaquinta,1983,DEC,red,02/03/2014,646,4649,3,3
7,Rashid Magomedov,2158,Tony Martin,2159,DEC,blue,11/05/2016,646,4651,3,3
8,Gasan Umalatov,2160,Neil Magny,2047,DEC,red,02/03/2014,646,4652,3,3
9,Ali Bagautinov,2084,John Lineker,1966,DEC,blue,02/03/2014,646,4658,3,3


In [6]:
fights[fights.B_Name=='Israel Adesanya']

,B_Name,B_ID,R_Name,R_ID,winby,winner,Date,Event_ID,Fight_ID,Max_round,Last_round
2004,Israel Adesanya,3051,Rob Wilkinson,3004,KO/TKO,blue,02/11/2018,857,7065,3,2
2204,Israel Adesanya,3051,Brad Tavares,1546,DEC,blue,07/09/2018,873,7258,5,5
